In [22]:
# -*- coding: utf-8 -*- 
import numpy as np
import scipy.sparse as sp
import pickle
import torch
import dgl
import pandas as pd
import mdtraj as md
from sklearn.preprocessing import label_binarize
import itertools

Using backend: pytorch


In [28]:
amino_acid = ['Gly','Ala','Val','Leu','Ile','Phe','Trp','Tyr','Asp','Asn','Glu','Lys','Gln',
              'Met','Ser','Thr','Cys','Pro','His','Arg','UNK']
acid_low = ['G','A','V','L','I','F','W','Y','D','N','E','K','Q','M','S','T','C','P','H','R','X']

aa_dic = {}
for i in range(21):
    aa_dic[amino_acid[i].upper()] = acid_low[i]
print(aa_dic)

index=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W','X', 'Y']
acid_dict={}
for i in range(len(index)):
    acid_dict[index[i]]=label_binarize([i], classes=range(0,21))[0]
data=acid_dict
df=pd.DataFrame(data,index=index)
df

{'GLY': 'G', 'ALA': 'A', 'VAL': 'V', 'LEU': 'L', 'ILE': 'I', 'PHE': 'F', 'TRP': 'W', 'TYR': 'Y', 'ASP': 'D', 'ASN': 'N', 'GLU': 'E', 'LYS': 'K', 'GLN': 'Q', 'MET': 'M', 'SER': 'S', 'THR': 'T', 'CYS': 'C', 'PRO': 'P', 'HIS': 'H', 'ARG': 'R', 'UNK': 'X'}


,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,X,Y
A,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
K,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
L,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def res_distance(residues_top,res_len):
    # residues_distance_unit = 'nanometers'
    res_index=range(res_len)
    residues_distance = md.compute_distances(residues_top,itertools.combinations(res_index,2))
    n=np.zeros((res_len,res_len))
    count=0
    #print(residues_distance)
    
    for i in range(0,res_len):
        for j in range(i+1,res_len):
        
            n[i][j]=residues_distance[0][count]
            n[j][i]=n[i][j]
            count+=1
    return n
def add_dis(n):
    res=[]
    rnge=len(n)
    for i in range(rnge):
        d={}
        for j in range(rnge):
            d[j]=n[i][j]
        
        s3 = sorted(d.items(), key=lambda d:d[1], reverse=False)
        for k in range(rnge):
            if s3[k][1]>0.8: #0.8nm=8Å
                flag=k
                break
        s3=s3[:20]
        res.append(s3)   
    return(res)

def cos_angle(xyz1,xyz2):
    angle1 = np.dot(xyz1 / 10, xyz2 / 10) / (np.linalg.norm(xyz1 / 10) * (np.linalg.norm(xyz2 / 10)))
    return angle1

In [32]:
graph = []
for pdb_ in predict:
    traj = md.load('/home/dldx/gin/model/'+pdb_+'.B99990001.pdb')
    print(pdb_)
    residues_top=traj[0]
    residues_index = residues_top.topology.select('name CA')
    res_len=len(residues_index)

    feat_num=21#结点特征维度
    feat = np.zeros((res_len, feat_num))

    G = dgl.DGLGraph()
    G.add_nodes(res_len)

    #计算节点距离
    n=res_distance(residues_top,res_len)
    res=add_dis(n)
    edge_w = []
    angle_w = []
    for index1 in range (res_len):
        e= res[index1]
        xyz1=residues_top.xyz[0, residues_index[index1],:]
        for t in e:
            G.add_edge(index1, t[0])
            edge_w.append(t[1])
            xyz2=residues_top.xyz[0, residues_index[t[0]],:]    
            angle1=cos_angle(xyz1,xyz2)   
            angle_w.append(angle1)
        resname=str(residues_top.topology.residue(index1))
        print(resname)
        feat[index1][0:21] = np.array(df.loc[aa_dic[resname[:3]]].values[0:],dtype=np.float64)
    edge_w = torch.tensor(edge_w).unsqueeze(1)
    angle_w = torch.tensor(angle_w).unsqueeze(1)
    angle_w=angle_w.double()

    #边的特征拼接
    edge_w = torch.cat([edge_w, angle_w], dim=1)
    G.edata["w"] = edge_w      

    feature = normalize(sp.csr_matrix(feat))
    feature = torch.tensor(np.array(feature.todense()))
    G.ndata["feat"] = feature
    G=G.to('cuda:0')
    if pdb_ in name1:
        label=0
    elif pdb_ in name2:
        label=1
    elif pdb_ in name3:
        label=2
    graph.append((G, label,str(label)+'-'+pdb_))
    print('label,pdb_: ',label, pdb_)
dataset_name = "data_predict.p"
with open(dataset_name, 'wb') as f:
    pickle.dump(graph, f)

Q8JDL4
LEU1
TRP2
VAL3
THR4
VAL5
TYR6
TYR7
GLY8
VAL9
PRO10
VAL11
TRP12
LYS13
ASP14
ALA15
GLU16
THR17
THR18
LEU19
PHE20
CYS21
ALA22
SER23
ASN24
ALA25
LYS26
ALA27
TYR28
GLY29
THR30
GLU31
VAL32
UNK33
ASN34
ILE35
TRP36
ALA37
THR38
HIS39
ALA40
CYS41
VAL42
PRO43
THR44
ASP45
PRO46
ASN47
PRO48
GLN49
GLU50
ILE51
ASN52
LEU53
GLU54
ASN55
VAL56
THR57
GLU58
GLU59
PHE60
ASN61
MET62
TRP63
LYS64
ASN65
ASN66
MET67
VAL68
GLU69
GLN70
MET71
HIS72
THR73
ASP74
ILE75
ILE76
SER77
LEU78
TRP79
ASP80
GLN81
GLY82
LEU83
LYS84
PRO85
CYS86
VAL87
LYS88
LEU89
THR90
PRO91
LEU92
CYS93
VAL94
THR95
LEU96
ASP97
CYS98
TYR99
ASN100
VAL101
THR102
LYS103
SER104
ASP105
LYS106
ILE107
THR108
LYS109
ASP110
MET111
GLN112
GLU113
GLU114
ILE115
LYS116
ASN117
CYS118
SER119
PHE120
ASN121
ILE122
THR123
THR124
GLU125
LEU126
ARG127
ASP128
LYS129
LYS130
GLN131
LYS132
VAL133
HIS134
SER135
LEU136
PHE137
TYR138
ARG139
LEU140
ASP141
VAL142
VAL143
PRO144
MET145
GLY146
GLY147
LYS148
ASN149
ASP150
SER151
GLN152
TYR153
ARG154
LEU155
ILE156
ASN157
CY